In [3]:
from nltk.corpus import senseval
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
import numpy as np

Using TensorFlow backend.


In [245]:
instances = senseval.instances('interest.pos')

In [246]:
size = int(len(instances)*0.1)

In [247]:
train_set, test_set = instances[size:], instances[:size]

In [248]:
train_set[1]

SensevalInstance(word='interest-n', position=12, context=[('the', 'DT'), ('idea', 'NN'), ('is', 'VBZ'), ('to', 'TO'), ('have', 'VB'), ('money', 'NN'), ('rolling', 'VBG'), ('over', 'IN'), ('each', 'DT'), ('year', 'NN'), ('at', 'IN'), ('prevailing', 'VBG'), ('interest', 'NN'), ('rates', 'NNS'), ('.', '.')], senses=('interest_6',))

In [244]:
>>> for inst in senseval.instances('interest.pos')[:10]:
...     p = inst.position
...     left = ' '.join(w for (w,t) in inst.context[p-2:p])
...     word = ' '.join(w for (w,t) in inst.context[p:p+1])
...     right = ' '.join(w for (w,t) in inst.context[p+1:p+3])
...     senses = ' '.join(inst.senses)
...     print('%20s |%10s | %-15s -> %s' % (left, word, right, senses))

         declines in |  interest | rates .         -> interest_6
  indicate declining |  interest | rates because   -> interest_6
       in short-term |  interest | rates .         -> interest_6
                 4 % |  interest | in this         -> interest_5
        company with | interests | in the          -> interest_5
              , plus |  interest | .               -> interest_6
             set the |  interest | rate on         -> interest_6
              's own |  interest | , prompted      -> interest_4
       principal and |  interest | is the          -> interest_6
        increase its |  interest | to 70           -> interest_5


In [251]:
sent = []
target = []
intances = ['hard.pos', 'interest.pos', 'line.pos', 'serve.pos']
for i in intances:
    for inst in senseval.instances(i)[:10]:
        p = inst.position
        alls = ' '.join(w for (w,t) in inst.context)
        sent.append(alls)
        #print(inst.senses[0])
        if inst.senses[0]=='interest_6':
            target.append('interest6')
        elif inst.senses[0]=='interest_5':
            target.append('interest5')
        elif inst.senses[0]=='interest_4':
            target.append('interest4')
        else:
            target.append(inst.senses[0])

        #print(inst.senses[0])
        #print(alls)
    


In [272]:
one_hot(intances,4)

AttributeError: 'list' object has no attribute 'lower'

In [273]:
text_to_word_sequence(sent)

AttributeError: 'list' object has no attribute 'lower'

In [66]:
#print(result)

['i', 'think', 'it', 'was', 'hard', 'for', 'the', 'saints', 'to', 'concentrate', 'tackle', 'steve', 'wallace', 'said', 'happy', 'the', 'superdome', 'dwellers', 'could', 'get', 'a', 'taste', 'of', 'their', 'own', 'medicine']


In [62]:
len(result)

26

In [264]:
result_idx = []
result_idy=[]

for s,t in zip(sent,target):
    result_idx.append(one_hot(s,50))
    
    result_idy.append(one_hot(t,50))


In [265]:
result_idy

[[31],
 [31],
 [31],
 [31],
 [31],
 [31],
 [31],
 [31],
 [31],
 [31],
 [25],
 [25],
 [25],
 [24],
 [24],
 [25],
 [25],
 [13],
 [25],
 [24],
 [39],
 [39],
 [39],
 [39],
 [39],
 [39],
 [39],
 [39],
 [39],
 [39],
 [46],
 [46],
 [46],
 [46],
 [46],
 [46],
 [46],
 [46],
 [46],
 [46]]

In [255]:
target

['HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'interest6',
 'interest6',
 'interest6',
 'interest5',
 'interest5',
 'interest6',
 'interest6',
 'interest4',
 'interest6',
 'interest5',
 'cord',
 'cord',
 'cord',
 'cord',
 'cord',
 'cord',
 'cord',
 'cord',
 'cord',
 'cord',
 'SERVE10',
 'SERVE10',
 'SERVE10',
 'SERVE10',
 'SERVE10',
 'SERVE10',
 'SERVE10',
 'SERVE10',
 'SERVE10',
 'SERVE10']

In [270]:
for i in range(40):
    for _ in range(6):
        result_idy[i].append(0)

In [188]:
#X = pad_sequences(result_idx)
#y = pad_sequences(result_idy)
#X = X.reshape(5,1,30)
#y = y.reshape(5,1,50)

In [269]:
len(result_idy)

40

In [275]:
pad_sequences(result_idy, maxlen=20)

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 31,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0

In [189]:
X = pad_sequences(result_idx).reshape(5,1,30)
y = pad_sequences(result_idy).reshape(5,1,30)

In [197]:
# Define Placeholders
#input_sequence = Input((text_max_length,))
#question = Input((ques_max_length,))
model = Sequential()
model.add(LSTM(units=50,input_shape=X.shape[1:],return_sequences=True, kernel_initializer='glorot_normal', recurrent_initializer='glorot_normal', activation='sigmoid'))
model.add(LSTM(units=50,return_sequences=True, kernel_initializer='glorot_normal', recurrent_initializer='glorot_normal', activation='sigmoid'))
model.add(LSTM(units=7,return_sequences=True, kernel_initializer='glorot_normal', recurrent_initializer='glorot_normal', activation='sigmoid'))
model.compile(loss ='cosine_proximity',optimizer='adam',metrics=['accuracy'])

In [198]:
model.fit(X, y, epoch=10,validation_split=0.1)

C:\ProgramData\Anaconda3\lib\site-packages\keras\models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 4 samples, validate on 1 samples
Epoch 1/10
4/4 [==============================] - 5s 1s/step - loss: -0.1958 - acc: 0.0000e+00 - val_loss: -0.1993 - val_acc: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 0s 3ms/step - loss: -0.1993 - acc: 0.0000e+00 - val_loss: -0.2027 - val_acc: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 0s 3ms/step - loss: -0.2028 - acc: 0.0000e+00 - val_loss: -0.2062 - val_acc: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 0s 5ms/step - loss: -0.2063 - acc: 0.0000e+00 - val_loss: -0.2097 - val_acc: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 0s 3ms/step - loss: -0.2099 - acc: 0.0000e+00 - val_loss: -0.2133 - val_acc: 0.0000e+00
Epoch 6/10
4/4 [==============================] - 0s 5ms/step - loss: -0.2135 - acc: 0.0000e+00 - val_loss: -0.2170 - val_acc: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 3ms/step - loss: -0.2173 - acc: 1.0000 - val_loss: -0.2208 - val_acc: 1.0000
Epoch

In [194]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 1, 50)             16200     
_________________________________________________________________
lstm_21 (LSTM)               (None, 1, 50)             20200     
_________________________________________________________________
lstm_22 (LSTM)               (None, 1, 50)             20200     
Total params: 56,600
Trainable params: 56,600
Non-trainable params: 0
_________________________________________________________________


In [9]:

# define documents
docs = ['Well done shyam genius',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]


In [10]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[8, 27, 33, 9], [5, 42], [31, 14], [15, 42], [29], [44], [8, 14], [25, 5], [8, 42], [31, 14, 27, 11]]


In [17]:
one_hot('Well done SHyam genius',50)

[8, 27, 33, 9]